In [ ]:
# Jetson Camera Toolkit - 单摄像头预览
import sys
import os

# 添加 utils 目录到路径
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath("."))))

import cv2 as cv
import threading
import ipywidgets as widgets
from IPython.display import display

from camera import VideoSource, build_csi_pipeline

In [ ]:
# UI 控件配置
button_layout = widgets.Layout(width='250px', height='50px', align_self='center')
output = widgets.Output()

# 退出按钮
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)

# 图像显示控件
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))

# 布局
controls_box = widgets.VBox([imgbox, exit_button], layout=widgets.Layout(align_self='center'))

# 全局状态
running = True

def exit_button_Callback(value):
    global running
    running = False
    with output:
        print("退出...")

exit_button.on_click(exit_button_Callback)

In [ ]:
# 摄像头配置
CSI_SENSOR_ID = 0  # 修改为你的 CSI 摄像头 ID
WIDTH = 640
HEIGHT = 480
FPS = 30

# 使用 toolkit 的 VideoSource
video_src = VideoSource(
    f"csi://{CSI_SENSOR_ID}",
    width=WIDTH,
    height=HEIGHT,
    fps=FPS
)

# 图像捕获函数
def capture_frames():
    global running
    
    if not video_src.open():
        with output:
            print("无法打开摄像头")
        return
    
    with output:
        print(f"摄像头已打开: {WIDTH}x{HEIGHT} @ {FPS}fps")
    
    while running:
        try:
            ret, frame = video_src.read()
            if not ret:
                break

            # 更新图像显示
            imgbox.value = cv.imencode(".jpg", frame)[1].tobytes()

        except KeyboardInterrupt:
            break
    
    video_src.release()
    with output:
        print("摄像头已释放")

# 显示控件
display(controls_box, output)

# 在新线程中捕获图像
threading.Thread(target=capture_frames, daemon=True).start()